<a href="https://colab.research.google.com/github/LuisFilipeAraujo/J.E.D.I-AI/blob/main/Treinamento_IA_Jedi_4_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = ':https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4543007%2F7772190%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240415%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240415T171253Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8f3c5335fca91a2d8772d46d90d81deb645edf5e54cca882f0b0eeb83922b970c64ca1e7ba8d030b98fbeb22578f42173c718f7aca57a9257ab4712adffb9509838b0d6e33240f2c9698bec57774f1b36ebe4e701b053803a8aa8b46b6d09ffbcc2bb4634e8de9f6403465f7928987893135b7ead544cbbf09118737266c8f2cfb9b43e2f8640b9f4ce1ba683dd734d016d3cbe6771d3270a90363bace6e32677712f26f7f7b24e10112e641478ac8b8548fd4c44ab039b0ae451b4e451f322fb282120998490d92bddf227eb0160fa10e7a8e0844d6d55d51a079cd001fa7e3e5b3b82d772ce6ac06ecd669d071768e215b0350882f70aedcc40fcd38741484'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Importing necessary Modules

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

# Loading the data and further cleaning

In [ ]:
raw_df = pd.read_csv('/kaggle/input/Popular_Spotify_Songs.csv', encoding = 'latin-1')
raw_df.head()

In [ ]:
raw_df.shape

In [ ]:
raw_df.info()

In [ ]:
raw_df.describe()

In [ ]:
mask = raw_df['streams'].str.isnumeric()

# '~' : negação
raw_df.loc[~mask]

In [ ]:
print("antes", raw_df.shape)
raw_df = raw_df.loc[mask]

print("depois", raw_df.shape)

raw_df.streams = raw_df.streams.astype('int64')

In [ ]:
df = raw_df.copy()

In [ ]:
df.columns

In [ ]:
df['artist(s)_name'].unique()[:10]

In [ ]:
# importancia da normalização
'J. Cole' in (df['artist(s)_name'].values)

In [ ]:
# normalizar df['artist(s)_name']

In [ ]:
# pegar artista principal da musica
temp = df['artist(s)_name'].str.split(',').str[0]
df['artist(s)_name'] = temp
df.rename(columns={'artist(s)_name':'artist'}, inplace=True)
df['artist'].head()

In [ ]:
df.head()

In [ ]:
# tratamento de data

# Passando dia da semana de numérico para nome do dia;
#     dependencia de ano e mes
temp = pd.DataFrame({'year':df.released_year, 'month':df.released_month, 'day':df.released_day})
temp = pd.to_datetime(temp).dt.day_name()
df.released_day = temp
df.head()

# Exploratory Data Analysis

#### Here we will explore the data and try to gain as much insight as possible

## No. of Unique Artist

In [ ]:
# difentes formas de fazer a mesma coisa
len(df['artist'].unique())

In [ ]:
df['artist'].unique().shape[0]

In [ ]:
df['artist'].nunique()

Their are total 458 unique artist in the dataset

## Number of Songs released (1980-2022)

In [ ]:
ax = sns.histplot(df['released_year'], binrange=(1980, 2022), discrete=True)
ax.set(xlabel='Year', ylabel='Count')
ax.set_title('Total Songs Released Per Year (1980 - 2024)')
plt.show()

Most of the songs in dataset were released in year 2022

## Trend of Song Release Month

In [ ]:
ax = sns.histplot(df['released_month'], discrete=True)
ax.set(xlabel='Month', ylabel='Count')
ax.set_title('Songs Released Group-By Months')
ax.set_xticks(range(1, 13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.show()

January saw the most released of the songs followed by May. August witness least song release.

## Trend of Song release Day

In [ ]:
ax = sns.histplot(df['released_day'])
ax.set(xlabel='Day', ylabel='Count')
ax.set_title('Songs Released Group-By Day')
plt.xticks(rotation = 30)
plt.show()

Most of the songs were released on Friday. This can be because people are more likely to stream music on weekends when they are free.

## Top 10 Artist with most song released

In [ ]:
temp = df.artist.value_counts().to_frame().head(10).iloc[:, 0]
temp_df = pd.DataFrame({'artist':temp.index, 'count':temp})

ax = sns.barplot(temp_df, y = 'artist', x = 'count')
ax.set(xlabel='Artist', ylabel='Total Songs')
ax.set_title('Artist with most songs')

plt.show()

Taylor Swift have the most songs released, followed by The Weeknd and Spanish Singer Bad Bunny.

## Beats Per Minute (BPM)

In [ ]:
ax = sns.histplot(df, x='bpm', bins=14)
ax.set_title('Beats per Minute (BPM) in songs')
plt.show()

120 BPM was the artist favourite. While 90,100,130 and 140 have equal preference among others.

## Top 10 songs according to their number in Playlist, Chart and Total Stream

In [ ]:
play = df.sort_values(by='in_spotify_playlists', ascending=False)
play = play[['track_name', 'in_spotify_playlists']][:10]

chart = df.sort_values(by='in_spotify_charts', ascending=False)
chart = chart[['track_name', 'in_spotify_charts']][:10]

stream = df.sort_values(by='streams', ascending=False)
stream = stream[['track_name', 'streams']][:10]

fig, axes = plt.subplots(3,1,figsize=(10,10))
plt.subplot(3,1,1)
ax1 = sns.barplot(play, y='track_name', x='in_spotify_playlists')
ax1.set_xlabel('count of spotify playlist')
ax1.set_title('Top songs according to playlist')

plt.subplot(3,1,2)
ax1 = sns.barplot(chart, y='track_name', x='in_spotify_charts')
ax1.set_xlabel('Charts')
ax1.set_title('Top songs according to Charts')


plt.subplot(3,1,3)
ax1 = sns.barplot(stream, y='track_name', x='streams')
ax1.set_xlabel('Total Streams')
ax1.set_title('Top songs according to Streams')

plt.tight_layout()
plt.show()

"Get Lucky - Radio Edit" was the most saved songs on chart, while "Blinding Lights" was the most streamed song. "Seven (feat. Latto)" was the Chart favourite song.

## Top 10 artist according to playlist, chart and total streams

In [ ]:
play = df.sort_values(by='in_spotify_playlists', ascending=False)
play = play[['artist', 'in_spotify_playlists']][:10]

chart = df.sort_values(by='in_spotify_charts', ascending=False)
chart = chart[['artist', 'in_spotify_charts']][:10]

stream = df.sort_values(by='streams', ascending=False)
stream = stream[['artist', 'streams']][:10]

fig, axes = plt.subplots(3,1,figsize=(10,10))
plt.subplot(3,1,1)
ax1 = sns.barplot(play, y='artist', x='in_spotify_playlists')
ax1.set_xlabel('count of spotify playlist')
ax1.set_title('Top artist according to playlist')

plt.subplot(3,1,2)
ax1 = sns.barplot(chart, y='artist', x='in_spotify_charts')
ax1.set_xlabel('Charts')
ax1.set_title('Top artist according to Charts')


plt.subplot(3,1,3)
ax1 = sns.barplot(stream, y='artist', x='streams')
ax1.set_xlabel('Total Streams')
ax1.set_title('Top artist according to Streams')

plt.tight_layout()
plt.show()

"Pharell Williams", "Latto" and "The Weeknd" dominated the playlist, chart and streams game respectively

## Top 10 artist by stream and their danceability, energy, valence and liveliness

In [ ]:
topten = df.sort_values(by='streams', ascending=False)
topten = topten[['artist', 'danceability_%', 'valence_%', 'energy_%', 'liveness_%', 'streams']][:5]
topten = pd.melt(topten, id_vars=['artist'], value_vars=['danceability_%', 'valence_%', 'energy_%', 'liveness_%'], var_name='type_of_%', value_name='value')

ax = sns.barplot(x='artist', y='value', hue='type_of_%', data=topten)
ax.legend(bbox_to_anchor=(1, 1))
ax.set(xlabel='Artist', ylabel='%')
ax.set_title('Top 5 Artist by streams and Their Vibes')

plt.show()

* "Ed Sheeran" and "Post Malone" songs are more cheerful or happy than rest of the artist (according to valency)
* "Tones and I" have to compete with "Ed Sheeran" for danceability of their songs
* "The Weeknd" have the most energetic songs
* All artist's songs are low on liveness


## Top 10 songs and their relation with danceability, valence, energy and liveness

In [ ]:
topten = df.sort_values(by='streams', ascending=False)
topten = topten[['track_name', 'danceability_%', 'valence_%', 'energy_%', 'liveness_%']][:5]
topten = topten.replace('Sunflower - Spider-Man: Into the Spider-Verse', 'Sunflower').set_index('track_name')

ax = sns.lineplot(data=topten, markers=True)
ax.legend(bbox_to_anchor=(1, 1))
plt.xticks(rotation=10)
plt.title("Top 5 Songs according to number of Streams", fontsize = 12, color = 'darkblue')
ax.set_ylabel('Percentage')
ax.set_xlabel('Track Name')

plt.show()

* "Shape of You" and "Sunflower" are more positive songs
* "Dance Monkey" and "Shape of You" are best songs to dance on
* "Blinding Lights" is the most energetic song